In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv

def unit_vec(vec):
  return vec / np.linalg.norm(vec)

# cross matrix allows a cross product to be performed with matrix multiplication
# vec is a numpy array 1d at least 3 dims
def cross_mat(vec):
  mat = np.array([[0, -vec[2], vec[1]],
                  [vec[2], 0, -vec[0]],
                  [-vec[1], vec[0], 0]])
  return mat


# axis is given as a 1d numpy array
# angle in radians
def axis_angle_to_q(axis, angle):
  # first unitize the axis
  axis_unit = unit_vec(axis)

  quat = np.array([axis_unit[0]*np.sin(angle/2), axis_unit[1]*np.sin(angle/2), axis_unit[2]*np.sin(angle/2), np.cos(angle/2)])

  return quat

# Gibbs vector is given as 1d numpy array
# no need to unitize gibbs vector (it is not a unit vector)
def gibbs_to_q(gibbs_vec):
  scale = 1 / np.sqrt(1 + np.dot(gibbs_vec, gibbs_vec))

  quat_setup = np.array([gibbs_vec[0], gibbs_vec[1], gibbs_vec[2], 1])

  return scale * quat_setup

# quaternion is a 4 element 1d numpy array
def quat_to_dcm(quat):
  #unitize first
  q = unit_vec(quat)

  q4 = q[3]
  qv = np.array([q[0], q[1], q[2]])

  coef1 = q4*q4 - np.inner(qv, qv)

  coef2 = 2 * np.outer(qv, qv)

  coef3 = 2*q4*cross_mat(qv)

  dcm = coef1 * np.identity(3) + coef2 - coef3
  return dcm


# Solving Problem 3 HERE----------------------------------

q_true = unit_vec(np.array([0.0485, 0.5932, 0.3781, 0.7091]))

q_est_1 = axis_angle_to_q(np.array([0.0769, 0.8436, 0.5314]), 1.5521)

q_est_2 = gibbs_to_q(np.array([0.0769, 0.8311, 0.5507]))


c_ti = quat_to_dcm(q_true)

c_ei_1 = quat_to_dcm(q_est_1)
c_ei_2 = quat_to_dcm(q_est_2)

# unitizing r_init as it is a direction
r_init = np.array([[-0.4846], [0.2546], [-0.8369]])
r = unit_vec(r_init)

b_true = c_ti @ r
b_e_1 = c_ei_1 @ r
b_e_2 = c_ei_2 @ r

dir_err_1 = np.degrees(np.arccos(b_e_1.T[0] @ b_true))[0]
dir_err_2 = np.degrees(np.arccos(b_e_2.T[0] @ b_true))[0]

grand1 = (np.trace(c_ti @ c_ei_1.T) - 1) / 2
grand2 = (np.trace(c_ti @ c_ei_2.T) - 1) / 2

att_err_1 = np.degrees(np.arccos(grand1))
att_err_2 = np.degrees(np.arccos(grand2))

print('The direction error of the first estimate is:')
print(dir_err_1, 'degrees')
print()
print('The direction error of the second estimate is:')
print(dir_err_2, 'degrees')
print()
print('The attitude error of the first estimate is:')
print(att_err_1, 'degrees')
print()
print('The attitude error of the second estimate is:')
print(att_err_2, 'degrees')

The direction error of the first estimate is:
0.7638254533576812 degrees

The direction error of the second estimate is:
1.0265752885010802 degrees

The attitude error of the first estimate is:
1.083546147848349 degrees

The attitude error of the second estimate is:
1.60752868779457 degrees


In [ ]:
# Solving Problem 4

import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
from numpy.linalg import eig
from numpy.linalg import det

# given inertial unit vectors and observed unit vectors
# ensure that they are unit vectors first!
# implement TRIAD method
# - should be just one equation

# implement q - method
# how to find q_opt?
# starts with Wahba relation
# replaces C with quaternion equation for DCM
# gain function is then a product of a transpose quaternion, a coefficient matrix, and a quaternion
# we name the coefficient matrix K
# later, we find q_opt is eigenvector associated with max eigenvalue

# so
# first find K using r and b

# CURRENT PROGRESS---------------

# then find eigenvalues of K and select max eigenvalue
# find eigenvector associated with this eigenvalue

# -----------------
# implementing TRIAD algorithm currently

def TRIAD(r_1, r_2, b_1, b_2):
  r_1_unit = unit_vec(r_1)
  r_2_unit = unit_vec(r_2)

  b_1_unit = unit_vec(b_1)
  b_2_unit = unit_vec(b_2)

  b_cross_comp2 = np.cross(b_1_unit, b_2_unit, axis=0)
  b_cross_comp3 = np.cross(b_1_unit, b_cross_comp2, axis=0)

  r_cross_comp2 = np.cross(r_1_unit, r_2_unit, axis=0)
  r_cross_comp3 = np.cross(r_1_unit, r_cross_comp2, axis=0)

  b_matrix = np.concatenate((b_1_unit, b_cross_comp2, b_cross_comp3), axis=1)
  r_matrix = np.concatenate((r_1_unit, r_cross_comp2, r_cross_comp3), axis=1)

  return b_matrix @ inv(r_matrix)

r_1 = np.array([[0.2150], [-0.4476], [-0.8680]])
r_2 = np.array([[-0.3295], [-0.1675], [0.9292]])

b_1 = np.array([[0.2201], [-0.3271], [-0.9190]])
b_2 = np.array([[-0.2165], [-0.3014], [0.9286]])

print('The computed attitude C using the TRIAD method is:')
print(TRIAD(r_1, r_2, b_1, b_2))
print()

# Now beginning the q-method to find q-opt

# Finding matrix K
def davenport_B(alphas, bs, rs):
  sum = np.zeros((3, 3))
  for i in range(0, len(alphas)):
    sum += alphas[i] * np.outer(bs[i], rs[i].T[0])
  return sum

#print(davenport_B([1/4, 1/4], [unit_vec(b_1), unit_vec(b_2)], [unit_vec(r_1), unit_vec(r_2)]))

def davenport_K(alphas, bs, rs):
  B = davenport_B(alphas, bs, rs)

  top_left = B + B.T - np.trace(B) * np.identity(3)

  top_right = np.zeros((3, 1))
  for i in range(0, len(alphas)):
    top_right += alphas[i] * np.cross(rs[i], bs[i], axis=0)

  bottom_left = top_right.T[0]

  bottom_right = np.trace(B)

  part1 = np.concatenate((top_left, top_right), axis=1)

  part2 = np.array([np.append(bottom_left, bottom_right)])

  total = np.concatenate((part1, part2), axis=0)

  return total

# computing the eigenvalues/eigenvectors
alphas = [1/4, 1/4]
bs = [unit_vec(b_1), unit_vec(b_2)]
rs = [unit_vec(r_1), unit_vec(r_2)]

eig_vals, eig_vecs = eig(davenport_K(alphas, bs, rs))

print('The eigenvalues of the K matrix according to the q-method is:')
print(eig_vals)
print()
print('We see that the max eigenvalue is 0.5, which corresponds to the final entry')
print('Finding the eigenvectors of the K matrix:')
print(eig_vecs)
print()
print('The eigenvector that corresponds to the optimal quaternion then, is:')
print(eig_vecs[:, 3])

The computed attitude C using the TRIAD method is:
[[ 0.98011936 -0.1801416   0.08209396]
 [ 0.18675062  0.97892021 -0.08169823]
 [-0.06568265  0.09548104  0.99324744]]

The eigenvalues of the K matrix according to the q-method is:
[-0.5        -0.40121099  0.40121099  0.5       ]

We see that the max eigenvalue is 0.5, which corresponds to the final entry
Finding the eigenvectors of the K matrix:
[[ 0.9596818   0.08713609 -0.2635031   0.04454505]
 [-0.07053255  0.99375854  0.0780225   0.03717012]
 [ 0.26424294 -0.05992422  0.95815962  0.09227626]
 [-0.06489739 -0.03550155 -0.08005471  0.99404186]]

The eigenvector that corresponds to the optimal quaternion then, is:
[0.04454505 0.03717012 0.09227626 0.99404186]
